# Испит по Основи на Вештачката Интелигенција - 1/2

**Датум**: 08.02.2024

## Проблем 1 - Кутии

Пред вас имате табла со 10 редови и 20 колони. Ова е светот во кој се движи агентот означен со знакот `🔷`. Неговата работа е да се движи до зададените полиња означени со `🟢`. За полесно означување, ќе ги користиме ознаките:
- Ознака за празно место: знакот `·` или бројот 0.
- Ознака за агент: знакот `🔷` или бројот 1.
- Ознака за поле целно поле: знакот `🟢` или бројот 2.

### Задача 1

#### Цел
- Целта е агентот `🔷` да стапне на кои било 2 зелени полиња `🟢` од понудените 3, така што ќе помине најмал пат.

#### Правила
- Агентот смее да се движи во сите осум насоки.
- Секој чекор на агентот е со должина 1 (едно поле).
- Агентот не смее да излезе надвор од таблата.

#### Прашања
1. **(10 поени)** Постави го овој проблем како проблем за пребарување.
    1. Како ќе ја дефнинираш состојбата за овој проблем? Објасни зошто ја избираш предложената дефиниција за состојба.
    
    1. Како ќе ја дефинираш почетната состојба, а како ќе ја дефинираш крајната состојба?
    
    1. Дали за проверка за крајна состојба ќе користиш состојба или функција за проверка дали е постигната целта?
    
    1. Која податочна структура ќе ја искористиш за да ја зачуваш информацијата за состојбата?
    
1. **(10 поени)** Објасни како ќе се одвива разгранувањето за дадена состојба, преку функцијата `expand_state()`?

1. **(30 поени)** Имплементирај го избраниот алгоритам користејќи пајтон, па отпечати барем едно можно решение на проблемот.

Таблата B, како објект од класата Board, служи за исцртување на решението. Можеш да ја користиш како помош при решавањето, а може и сам да си имплементираш решение за прикажување на состојбата на светот во која се наоѓа агентот. Ако ти треба нова табла, слободно направи нов објект од класата.

In [2]:
from exam_help import Board, generate_random_table
from collections import deque
import heapq

ROWS, COLS = 10, 20
B = Board(ROWS, COLS)
B



Board()

In [3]:
random_table = generate_random_table(n_rows=ROWS, n_cols=COLS, n_agents=1, n_goal_fields=3)
B.update(random_table)

In [4]:
## objasnuvanje
# sostojbata ke ja definirame kako triple (x, y, poseteni) lokacijata i frozen set od vrednosti za poseteni goals, a startnata sostojba se x_start y_start i posteni {}
# 

In [5]:
start = None
goals = []
for i in range(ROWS):
    for j in range(COLS):
        if random_table[i][j] == 1:
            start = (i, j)
        elif random_table[i][j] == 2:
            goals.append((i, j))
            
print("Startna pozicija: ", start)
print("Celi: ", goals)

Startna pozicija:  (7, 16)
Celi:  [(1, 0), (1, 11), (7, 7)]


In [6]:
pocetna_poz = None
goals = []
for i in range(ROWS):
    for j in range(COLS):
        if random_table[i][j] == 1:
            pocetna_poz = (i, j)
        elif random_table[i][j] == 2:
            goals.append((i, j))
            
print(pocetna_poz)
print(goals)

(7, 16)
[(1, 0), (1, 11), (7, 7)]


In [9]:
# BFS PO SIRINA
def bfs_search(board, start, goals):
    rows, cols = len(board), len(board[0])
    start_state = (start[0], start[1], frozenset())
    
    queue = deque([start_state])
    visited = set()
    visited.add(start_state)
    came_from = {start_state: None}
    broj_na_kombinacii = 0
    
    while queue:
        x, y, visited_goals = queue.popleft()
        
        if len(visited_goals) == 2:
            path = reconstruct_path(came_from, (x, y, visited_goals))
            return (path, broj_na_kombinacii)
        
        new_states = expand_state(x, y, visited_goals, board)
        for new_state in new_states:
            broj_na_kombinacii += 1
            if new_state not in visited:
                visited.add(new_state)
                queue.append(new_state)
                came_from[new_state] = (x, y, visited_goals)
    
    return None, broj_na_kombinacii

result = bfs_search(random_table, start, goals)
print("Најдено решение:", result)

Најдено решение: ([None, (7, 16, frozenset()), (6, 16, frozenset()), (5, 15, frozenset()), (4, 14, frozenset()), (3, 13, frozenset()), (2, 12, frozenset()), (1, 11, frozenset({(1, 11)})), (2, 11, frozenset({(1, 11)})), (3, 11, frozenset({(1, 11)})), (4, 10, frozenset({(1, 11)})), (5, 9, frozenset({(1, 11)})), (6, 8, frozenset({(1, 11)})), (7, 7, frozenset({(1, 11), (7, 7)}))], 1890)


In [10]:
# A* NACIN

import heapq
from exam_help import Board, generate_random_table

def heuristic(x, y, goals):
    return min(abs(x - gx) + abs(y - gy) for gx, gy in goals) 

def expand_state(x, y, visited_goals, board):
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, -1), (-1, 1), (1, 1)]
    rows, cols = len(board), len(board[0])
    new_states = []
    
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols:
            new_visited_goals = visited_goals.copy()
            if board[nx][ny] == 2:
                new_visited_goals = new_visited_goals | {(nx, ny)}
            
            new_state = (nx, ny, frozenset(new_visited_goals))
            new_states.append(new_state)
    
    return new_states

def a_star_search(board, start, goals):
    rows, cols = len(board), len(board[0])
    start_state = (start[0], start[1], frozenset())
    goal_count = 2
    
    open_set = []
    heapq.heappush(open_set, (0, start_state))
    g_score = {start_state: 0}
    f_score = {start_state: heuristic(start[0], start[1], goals)}
    came_from = {}
    kombinacii = 0
    
    while open_set:
        _, current = heapq.heappop(open_set)
        x, y, visited_goals = current
        
        if len(visited_goals) == goal_count:
            path = reconstruct_path(came_from, current)
            return (path, kombinacii)
        
        new_states = expand_state(x, y, visited_goals, board)
        for new_state in new_states:
            nx, ny, new_visited_goals = new_state
            tentative_g_score = g_score[current] + 1
            kombinacii += 1
            if new_state not in g_score or tentative_g_score < g_score[new_state]:
                came_from[new_state] = current
                g_score[new_state] = tentative_g_score
                f_score[new_state] = tentative_g_score + heuristic(nx, ny, goals)
                heapq.heappush(open_set, (f_score[new_state], new_state))
    
    return None, kombinacii

def reconstruct_path(came_from, current):
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.append(current)
    total_path.reverse()
    return total_path
path, combinations = a_star_search(random_table, start, goals)
print("Најден пат:", path)
print("Број на комбинации:", combinations)
print("Чекори за најбрз пат:", len(path) - 1)

Најден пат: [(7, 16, frozenset()), (6, 15, frozenset()), (5, 14, frozenset()), (4, 13, frozenset()), (3, 12, frozenset()), (2, 11, frozenset()), (1, 11, frozenset({(1, 11)})), (2, 10, frozenset({(1, 11)})), (3, 9, frozenset({(1, 11)})), (4, 8, frozenset({(1, 11)})), (5, 7, frozenset({(1, 11)})), (6, 7, frozenset({(1, 11)})), (7, 7, frozenset({(1, 11), (7, 7)}))]
Број на комбинации: 2516
Чекори за најбрз пат: 12
